# Springboard Data Science Career Track

## Capstone Project #2: Learning Neural Networks through Text Generation

## Extended Model #11: Further Increasing Number of Epochs of Word-Level Model

##### By Logan Larson

In [1]:
import numpy
from numpy import array
import string
import re
from random import randint
from pickle import load, dump
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Fu

In [2]:
# load text and convert to lowercase
filename = "AdventuresOfSherlockHolmes.txt"
raw_text = open(filename).read()
print(raw_text[:102])

To Sherlock Holmes she is always THE woman. I have seldom heard
him mention her under any other name. 


In [3]:
# convert to lowercase
raw_text = raw_text.lower()
# replace dashes
raw_text = raw_text.replace('--', '')
# split words by white space
tokens = raw_text.split()
# remove punctuation from each word
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
tokens = [re_punc.sub('', w) for w in tokens]
# remove any non-alphabetic tokens
tokens = [word for word in tokens if word.isalpha()]
print(tokens[:19])

['to', 'sherlock', 'holmes', 'she', 'is', 'always', 'the', 'woman', 'i', 'have', 'seldom', 'heard', 'him', 'mention', 'her', 'under', 'any', 'other', 'name']


In [4]:
len(tokens)

104202

In [5]:
seq_length = 8
words = sorted(list(set(tokens)))
token_to_int = dict((c, i) for i, c in enumerate(words))
dataX = []
dataY = []
n_tokens = len(tokens)
for i in range(0, n_tokens - seq_length, 1):
    seq_in = tokens[i:i + seq_length]
    seq_out = tokens[i + seq_length]
    dataX.append([token_to_int[token] for token in seq_in])
    dataY.append(token_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  104194


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(len(words))

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

print(X.shape[1], X.shape[2])

8 1


In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [8]:
# fit the model
model.fit(X, y, epochs=60, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/60
104194/104194 [==============================] - 98s 936us/step - loss: 6.6110

Epoch 00001: loss improved from inf to 6.61104, saving model to weights-improvement-01-6.6110.hdf5
Epoch 2/60
104194/104194 [==============================] - 101s 969us/step - loss: 6.4574

Epoch 00002: loss improved from 6.61104 to 6.45745, saving model to weights-improvement-02-6.4574.hdf5
Epoch 3/60
104194/104194 [==============================] - 107s 1ms/step - loss: 6.4470

Epoch 00003: loss improved from 6.45745 to 6.44698, saving model to weights-improvement-03-6.4470.hdf5
Epoch 4/60
104194/104194 [==============================] - 105s 1ms/step - loss: 6.4416

Epoch 00004: loss improved from 6.44698 to 6.44162, saving model to weights-improvement-04-6.4416.hdf5
Epoch 5/60
104194/104194 [==============================] - 106s 1ms/step - loss: 6.4357

Epoch 00005: loss improved from 6.44162 to 6.

In [9]:
# load the network weights
filename = "weights-improvement-60-3.7292.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
import sys

int_to_token = dict((i, c) for i, c in enumerate(words))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" your commonplace featureless crimes which are really puzzling "
the a i manner of the the stock i the the and buried i the planet of the the states and the mostly concerned to coronet the the the the holmes the the the and the repeat war entreaties disappeared the the cried the the reasoned and the lazily the guilty assertion shattered the hosmer roads marseilles prolonged absence described caused the advise snowclad the stretched floor of a the tobacco the gate of the the rabbit and the the and contemplative purveyor papers side gripping the a the chamber the figure of the fields the the wont unhappy must 
Done.


In [11]:
start2 = numpy.random.randint(576, len(dataX)-1)
pattern = dataX[start2]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" there he was still with a kind of "
the the coincidences to i the the the the eyford the the supplied a the lazily the bohemia and regurgitation of cornwall the waiting of the the of the the brightly dwelling the the the wheeled demand the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the the cover the the the 
Done.


In [12]:
start3 = numpy.random.randint(1291, len(dataX)-1)
pattern = dataX[start3]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" i do so want a little help why "
a and the i of the and at it i to world frenzy the alpha the the and the whose skylight following the i the the the recognize was the wagons the the the the the have of the tension i a i the the mto the results naturally he confessed the the the i of the the suit a the the hereford of the i amusing i the the a the corridor of the the precisely abutted drunkards narrative implicate the the and the ate the afternoon each forth i the the the the are and the i the 
Done.


In [13]:
start4 = numpy.random.randint(291, len(dataX)-1)
pattern = dataX[start4]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" consequences to me far from hushing the thing "
business comfortablelooking the the a and the method of answering the the of the is the a the murdering the the providing was kitchen purport the the suggestiveness a pitch pile expectancy the the the depressing the the the afterwards the lazily of the gold the the wednesday the permission to the chubb the the the of english illuminated the the pleased the law the the and it and the to the had please afterwards the the the the is the the the the county of the the sisters and sounded busy the the tied will be the scattered bred 
Done.


In [14]:
start5 = numpy.random.randint(2291, len(dataX)-1)
pattern = dataX[start5]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" every way that they should make their position "
the saucer the milk the the the and the mostly faced the coronet the the the have the a to the the rain a probing the sufficient punishment the i the the the and the securea war entreaties shook the of the you the the the the the justified of the the the a the the hereford of the i amusing i the the a the corridor of the the precisely abutted drunkards narrative implicate the the and the ate the afternoon each forth i the the the the are and the i the the i a the i the 
Done.
